### Import

In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Numpy version: 1.26.4
Scikit-learn version: 1.5.1
Pandas version: 1.5.3
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [ ]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train = missing_value_removal_function(train)
test = missing_value_removal_function(test)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


In [4]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [5]:
columns_fill_zero = [
    # 'PGD 시술 여부', 'PGS 시술 여부',
    '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부'
]
X[columns_fill_zero] = X[columns_fill_zero].fillna(0)
test[columns_fill_zero] = test[columns_fill_zero].fillna(0)

In [6]:
# 각 열의 결측값 개수 확인
missing_values_count = X.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)

결측값이 있는 열과 개수:
Series([], dtype: int64)


In [7]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('알 수 없음').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ '알 수 없음' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ '알 수 없음' 값을 포함하는 컬럼들:
['시술 당시 나이', '배란 유도 유형', '난자 기증자 나이', '정자 기증자 나이']


In [8]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('nan').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ 'nan' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ 'nan' 값을 포함하는 컬럼들:
[]


In [9]:
# NaN 값이 존재하는 컬럼 찾기
unknown_columns = X.columns[X.isna().any()].tolist()

unknown_columns

[]

In [ ]:
# Categorical(범주형) 칼럼 찾기
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

# 결과 출력
print("📌 Categorical(범주형) 칼럼 리스트:")
print(categorical_columns)


📌 Categorical(범주형) 칼럼 리스트:
['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이']


In [11]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [12]:
columns_to_drop = [
        "남성 주 불임 원인",
        "남성 부 불임 원인",
        "불임 원인 - 정자 농도",
        "불임 원인 - 정자 면역학적 요인",
        "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태",
        '정자 기증자 나이',
        '배란 유도 유형'
]
X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)  

In [13]:
# 각 열의 결측값 개수 확인
missing_values_count = X_train_encoded.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)


결측값이 있는 열과 개수:
Series([], dtype: int64)


In [14]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 55 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                256351 non-null  float64
 1   시술 시기 코드          256351 non-null  float64
 2   시술 당시 나이          256351 non-null  float64
 3   시술 유형             256351 non-null  float64
 4   특정 시술 유형          256351 non-null  float64
 5   배란 자극 여부          256351 non-null  int64  
 6   단일 배아 이식 여부       256351 non-null  float64
 7   착상 전 유전 검사 사용 여부  256351 non-null  float64
 8   착상 전 유전 진단 사용 여부  256351 non-null  float64
 9   여성 주 불임 원인        256351 non-null  int64  
 10  여성 부 불임 원인        256351 non-null  int64  
 11  부부 주 불임 원인        256351 non-null  int64  
 12  부부 부 불임 원인        256351 non-null  int64  
 13  불명확 불임 원인         256351 non-null  int64  
 14  불임 원인 - 난관 질환     256351 non-null  int64  
 15  불임 원인 - 남성 요인     256351 non-null  int64  
 16  불임 원인 - 배란 장애     25

In [15]:

X_train_encoded['배아 생성 주요 이유'].value_counts()

1    253108
0      3243
Name: 배아 생성 주요 이유, dtype: int64

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 (X_train_encoded & X_test_encoded)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용

# DataFrame 변환 (Feature 이름 유지)
feature_names = [f"Feature_{i}" for i in range(X_train_scaled.shape[1])]
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

# 상관 행렬 계산
correlation_matrix_train = X_train_scaled_df.corr()

# 다중 공선성이 높은 칼럼 찾기 (절대 상관 계수가 0.8 이상)
threshold = 0.8
high_corr_features = set()

for i in range(len(feature_names)):
    for j in range(i + 1, len(feature_names)):
        if abs(correlation_matrix_train.iloc[i, j]) > threshold:
            high_corr_features.add(feature_names[j])  # 공선성이 높은 컬럼 추가

# 다중 공선성이 높은 컬럼 제거
X_train_encoded = X_train_scaled_df.drop(columns=high_corr_features, errors='ignore')
X_test_encoded = X_test_scaled_df.drop(columns=high_corr_features, errors='ignore')


In [16]:
X_train_encoded

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_41,Feature_44,Feature_45,Feature_46,Feature_48,Feature_50,Feature_51,Feature_52,Feature_53,Feature_54
0,-1.732044,1.518507,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
1,-1.732031,1.020975,2.655919,0.158613,-0.962962,-1.836376,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
2,-1.732017,0.025910,-0.935575,0.158613,0.672585,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.773571
3,-1.732004,-0.471622,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
4,-1.731990,0.025910,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,1.731990,1.020975,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,1.142081
256347,1.732004,1.020975,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
256348,1.732017,0.025910,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
256349,1.732031,1.518507,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.103519,-0.11236,-0.178038,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-1.412122


### Train

In [17]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

**앙상블 및 Optuna**

In [18]:
import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [19]:
def optimize_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }
    model = XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
    return np.mean(scores)

In [20]:
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0)
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)

In [21]:
def optimize_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)


In [22]:
# Optuna 스터디 실행
print("Optimizing XGBoost...")
xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(optimize_xgboost, n_trials=30)
best_xgb_params = xgb_study.best_params

print("Optimizing LightGBM...")
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(optimize_lightgbm, n_trials=30)
best_lgbm_params = lgbm_study.best_params

print("Optimizing RandomForest...")
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(optimize_random_forest, n_trials=30)
best_rf_params = rf_study.best_params

[I 2025-02-15 23:03:04,240] A new study created in memory with name: no-name-792c8ccd-b3a1-40ac-8a44-c393ed529979


Optimizing XGBoost...


c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [23:03:04] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [23:03:05] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [23:03:07] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [23:03:08] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

Optimizing LightGBM...
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008005 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 764
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

[I 2025-02-15 23:05:54,631] Trial 0 finished with value: 0.7350168410492423 and parameters: {'n_estimators': 201, 'max_depth': 4, 'learning_rate': 0.08159408079067006, 'num_leaves': 48, 'subsample': 0.793905548546906}. Best is trial 0 with value: 0.7350168410492423.


...

[I 2025-02-15 23:07:40,258] Trial 28 finished with value: 0.7299659342888903 and parameters: {'n_estimators': 104, 'max_depth': 6, 'learning_rate': 0.1841295266307875, 'num_leaves': 30, 'subsample': 0.8418741615579556}. Best is trial 19 with value: 0.7366116210595639.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007259 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 764
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2025-02-15 23:07:43,630] Trial 29 finished with value: 0.7307793321560311 and parameters: {'n_estimators': 195, 'max_depth': 4, 'learning_rate': 0.2593205172736167, 'num_leaves': 41, 'subsample': 0.967140694058794}. Best is trial 19 with value: 0.7366116210595639.
[I 2025-02-15 23:07:43,631] A new study created in memory with name: no-name-3835f45a-9479-4d6b-a75a-b915c2ce76d2


Optimizing RandomForest...


[I 2025-02-15 23:08:40,213] Trial 0 finished with value: 0.7290740173259109 and parameters: {'n_estimators': 127, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7290740173259109.
[I 2025-02-15 23:09:42,675] Trial 1 finished with value: 0.7193829431621217 and parameters: {'n_estimators': 241, 'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7290740173259109.
[I 2025-02-15 23:12:09,273] Trial 2 finished with value: 0.7329744374891373 and parameters: {'n_estimators': 237, 'max_depth': 12, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.7329744374891373.
[I 2025-02-15 23:13:12,183] Trial 3 finished with value: 0.7143479585229267 and parameters: {'n_estimators': 295, 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.7329744374891373.
[I 2025-02-15 23:15:19,482] Trial 4 finished with value: 0.7332545819707568 and parameters: {'n

In [23]:
# 최적화된 모델 생성
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

In [24]:
# XGBoost 모델의 파라미터 확인
print("XGBoost Best Parameters:")
print(xgb_model.get_params())

# LightGBM 모델의 파라미터 확인
print("\nLightGBM Best Parameters:")
print(lgbm_model.get_params())

# RandomForest 모델의 파라미터 확인
print("\nRandomForest Best Parameters:")
print(rf_model.get_params())

XGBoost Best Parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6534129262787071, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'gamma': 4.192179038042475, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.11014950780428198, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 4, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 279, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.6751815632775964, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder':

In [25]:
# Soft Voting 앙상블
ensemble_model = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("lgbm", lgbm_model),
        ("rf", rf_model)
    ],
    voting="soft"
)

In [26]:
# 전체 데이터로 학습
ensemble_model.fit(X_train_encoded, y)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [00:00:33] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008764 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 764
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.6534129262787071,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            gamma=4.192179038042475,
                                            grow_policy=None,
                                            importance_type=None,
                                            interact...
                                            n_estimators=279, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.188225657868898,
                                             max_depth=3, n_estimators=107,
                                             num_leaves=15, random_state=42,
                                             subsample=0.6047794133513343)),
                             ('rf',
                              RandomForestClassifier(max_depth=12,
                                                     min_samples_leaf=3,
                                                     min_samples_split=7,
                                                     n_estimators=224,
                                                     random_state=42))],
                 voting='soft')

### Predict

In [27]:
pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [28]:
sample_submission = pd.read_csv('../../sample_submission.csv')
sample_submission['probability'] = pred_proba

In [29]:
sample_submission.to_csv('./Result/baseline_submit.csv', index=False)

In [30]:
pred_proba

array([0.02240862, 0.01687016, 0.15057783, ..., 0.45519076, 0.27236221,
       0.01414558])

In [31]:
unique, counts = np.unique(sample_submission['probability'], return_counts=True)
print(unique, counts)

[4.49215711e-04 4.83187075e-04 5.58406526e-04 ... 6.69323316e-01
 6.70454058e-01 6.84927437e-01] [1 1 1 ... 1 1 1]
